#### (1) 기본 예시: 프롬프트 + 모델 + 출력 파서

- 가장 기본적이고 일반적인 사용 사례 = **prompt 템플릿과 모델을 함께 연결하는 것**
- 예시: 각 나라별 수도를 물어보는 Chain을 생성 -> **작동 방식** 살펴보기


In [ ]:
# 기본 모듈 임포트
import os
from dotenv import load_dotenv

# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보 로드
load_dotenv()                   # true

In [ ]:
# 환경 변수 확인하기

# 마스킹 처리 함수 정의
def mask_key(key: str, visible_count: int = 2) -> str:
    if not key or len(key) <= visible_count:
        return '*' * len(key)
    return key[:visible_count] + '*' * (len(key) - visible_count)

# 환경변수 불러오기
api_key = os.getenv("GOOGLE_API_KEY")
if not api_key:
    raise ValueError("GOOGLE_API_KEY 환경 변수가 설정되지 않았습니다.")

# 마스킹된 형태로 출력
print(f"GOOGLE_API_KEY: {mask_key(api_key)}")           # GOOGLE_API_KEY: AI*************************************

---

In [3]:
# LangSmith 추적 설정 (https://smith.langchain.com)

"""
- !pip install -qU langsmith
- !pip install -qU langchain-teddynote
    -> 제미나이와 poetry와의 의존성 충돌로 langchain_teddy 설치 X 
    -> langsmith로 진행
"""
# LangSmith 추적을 위한 라이브러리 임포트
from langsmith import traceable         # @traceable 데코레이터 사용 시
from langsmith import Client            # 명시적으로 Client 객체를 다룰 때 사용 (여기서는 불필요하지만 임포트해도 무방)

In [ ]:
# LangSmith 환경 변수 확인

print("\n--- LangSmith 환경 변수 확인 ---")
langchain_tracing_v2 = os.getenv('LANGCHAIN_TRACING_V2')
langchain_project = os.getenv('LANGCHAIN_PROJECT')
langchain_api_key_status = "설정됨" if os.getenv('LANGCHAIN_API_KEY') else "설정되지 않음" # API 키 값은 직접 출력하지 않음

if langchain_tracing_v2 == "true" and os.getenv('LANGCHAIN_API_KEY') and langchain_project:
    print(f"✅ LangSmith 추적 활성화됨 (LANGCHAIN_TRACING_V2='{langchain_tracing_v2}')")
    print(f"✅ LangSmith 프로젝트: '{langchain_project}'")
    print(f"✅ LangSmith API Key: {langchain_api_key_status}")
    print("  -> 이제 LangSmith 대시보드에서 이 프로젝트를 확인해 보세요.")
else:
    print("❌ LangSmith 추적이 완전히 활성화되지 않았습니다. 다음을 확인하세요:")
    if langchain_tracing_v2 != "true":
        print(f"  - LANGCHAIN_TRACING_V2가 'true'로 설정되어 있지 않습니다 (현재: '{langchain_tracing_v2}').")
    if not os.getenv('LANGCHAIN_API_KEY'):
        print("  - LANGCHAIN_API_KEY가 설정되어 있지 않습니다.")
    if not langchain_project:
        print("  - LANGCHAIN_PROJECT가 설정되어 있지 않습니다.")

<small>

* 셀 출력
    * --- LangSmith 환경 변수 확인 ---
    * ✅ LangSmith 추적 활성화됨 (LANGCHAIN_TRACING_V2='true')
    * ✅ LangSmith 프로젝트: 'L*************'                           <- 마스킹 처리
    * ✅ LangSmith API Key: 설정됨
    *   -> 이제 LangSmith 대시보드에서 이 프로젝트를 확인해 보세요.

---

#### (2) 프롬프트 템플릿의 활용

- `PromptTemplate`

  - 사용자의 입력 변수를 사용하여 완전한 프롬프트 문자열을 만드는 데 사용되는 템플릿
  - 사용법
    - `template`: 템플릿 문자열 / 이 문자열 내 중괄호 `{}` = **변수**를 의미
    - `input_variables`: 중괄호 안에 들어갈 변수의 이름 -> **리스트**로 정의


- `input_variables`
  - `input_variables`: PromptTemplate에서 사용되는 **변수의 이름을 정의하는 리스트**

In [5]:
# LangChain 및 Google GenAI 모델 관련 모듈 임포트
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI           # Google GenAI 임포트

# from google import genai                                          # Gen AI SDK로 스트리밍 임포트
# client  = genai.Client()

#### (3) LCEL

- **프롬프트 템플릿, 모델, 출력 파서 정의 및 체인** 구성
  
  - `ChatGoogleGenerativeAI`모델 -> LCEL 체인에 통합될 수 있도록 설계됨
    - `generate_content_stream` 같은 스트리밍 API 호출 -> 결과 가져올 수 있음
    - LangChain의 `AIMessageChunk` 객체 형태로 반환 -> 다음 구성 요소 (예: `StrOutputParser`)로 전달 가능
  
  - **LCEL Chain (Prompt + Model + Output Parser)**
    - **LangChain**에서 제공하는 **Runnable 객체들을 연결하는 방식**
  
    ![lcel.png](./images/lcel.png)

      ```
      chain = prompt | model | output_parser
      ```
    - LCEL 사용 ->  다양한 구성 요소를 단일 체인으로 결합
    - `|` 기호는 [unix 파이프 연산자](<https://en.wikipedia.org/wiki/Pipeline_(Unix)>)와 유사
      - 서로 다른 구성 요소를 연결하고 한 구성 요소의 출력을 다음 구성 요소의 입력으로 전달

    - 사용자 입력 -> 프롬프트 템플릿 -> 모델 -> 각 구성 요소를 개별적으로 살펴보면 무슨 일이 일어나고 있는지 이해 가능

    
    - **Model 부분**
      - `langchain_openai.ChatOpenAI` or `langchain_google_genai.ChatGoogleGenerativeAI`와 같이 `LangChain이` 제공하는 **래퍼(Wrapper) 모델 객체**여야 함
      - 이 래퍼 모델들은 내부적으로 `google.genai` 같은 **SDK를 호출하고 스트리밍 처리를 담당**

In [ ]:
# 프롬프트 템플릿, 모델, 출력 파서 정의 및 체인 구성

print("\n--- LangChain 체인 설정 ---")

template = "{topic}에 대해 쉽게 설명해주세요."                            # 템플릿 정의

prompt = PromptTemplate.from_template(template)                     # 프롬프트를 프롬프트템플릿 객체로 생성

try:
    model = ChatGoogleGenerativeAI(                                 # 모델 호출
        model="gemini-1.5-flash",
        temperature=0.1,
    )
    print("✅ Google GenAI 모델 초기화 성공.")
except Exception as e:                                              # 디버깅 메시지
    print(f"❌ Google GenAI 모델 초기화 실패: {e}")
    print("  -> GOOGLE_API_KEY 환경 변수가 올바르게 설정되었는지 확인하세요.")
    
# 출력 파서
output_parser = StrOutputParser()

# 체인 구성
chain = prompt | model | output_parser                              # 프롬프트, 모델, 출력 파서 연결 -> 체인 구성
print("✅ LangChain LCEL 체인 구성 완료.")

<small>

* 셀 출력
    * --- LangChain 체인 설정 ---
    * ✅ Google GenAI 모델 초기화 성공.
    * ✅ LangChain LCEL 체인 구성 완료.

---

* 스트리밍 출력 준비

In [7]:
# 스트리밍 출력을 위한 헬퍼 함수 (재사용)

def stream_and_print_response(answer_iterator):
    for chunk in answer_iterator:
        print(chunk, end="", flush=True)
    print()

---

#### (4) **invoke()** 호출

- `python 딕셔너리 형태`로 입력값 전달  -> (키: 값)
- `invoke()` 함수 호출 -> 입력값을 전달

In [10]:
# input 딕셔너리에 주제를 '인공지능 모델의 학습 원리'으로 설정
input_data = {"topic": "인공지능 모델의 학습 원리"}

In [ ]:
# chain에 파이프 연산자(|)로 model 객체 + invoke 메서드 연결하여 input으로 전달
print("\n--- invoke() 호출 결과 ---")
try:
    response = chain.invoke(input_data)
    print(response)
    print("✅ invoke() 호출 성공.")
except Exception as e:
    print(f"❌ invoke() 호출 중 오류 발생: {e}")
    print("  -> 모델 응답, API 키, 네트워크 연결, 또는 프롬프트 내용을 확인해 보세요.")

# 스트리밍 함수 호출
print("\n--- stream() 호출 결과 (직접 스트리밍 출력) ---")
try:
    answer_stream = chain.stream(input_data)
    stream_and_print_response(answer_stream)
    print("✅ stream() 호출 성공.")
except Exception as e:
    print(f"❌ stream() 호출 중 오류 발생: {e}")
    print("  -> 스트리밍 처리, 모델 응답, API 키, 네트워크 연결 등을 확인해 보세요.")

<small>

* 셀 출력
  
    * --- invoke() 호출 결과 ---
      * 인공지능 모델의 학습 원리는 마치 어린아이가 배우는 것과 비슷합니다.  아이가 "강아지"라는 단어와 강아지 사진을 여러 번 반복해서 보면서 "강아지"라는 단어가 무엇을 의미하는지 배우는 것처럼, AI 모델도 데이터를 통해 학습합니다.

        * **1. 데이터 제공:**  AI 모델에게는 수많은 데이터(예: 강아지 사진, 고양이 사진, 설명 등)를 제공합니다. 이 데이터는 모델이 학습할 "교과서"와 같습니다.

        * **2. 모델의 예측:** 모델은 제공된 데이터를 바탕으로 스스로 패턴을 찾아내려고 합니다.  처음에는 아무것도 모르기 때문에 틀린 예측을 많이 할 것입니다. 예를 들어, 강아지 사진을 보고 "고양이"라고 예측할 수도 있습니다.

        * **3. 오류 수정 (학습):** 모델의 예측이 틀렸다면, 정답과 비교하여 오류를 파악합니다.  이 오류를 바탕으로 모델 내부의 매개변수(가중치와 편향)를 조정합니다.  이 과정을 통해 모델은 다음에는 더 정확한 예측을 할 수 있도록 "학습"합니다.  마치 아이가 틀린 답을 했을 때, 부모가 바른 답을 알려주고 다시 시도하게 하는 것과 같습니다.

        * **4. 반복:** 2번과 3번 과정을 수없이 반복합니다.  데이터를 많이 학습할수록, 오류를 수정할 기회가 많아지고, 모델의 정확도는 점점 높아집니다.  이 과정을 "훈련(Training)"이라고 합니다.

      * **간단한 비유:**

        * **모델:**  빈 용기
        * **데이터:**  물
        * **학습:**  물을 용기에 채우는 과정
        * **정확도:**  용기에 채워진 물의 양

      * 물을 많이 채울수록(데이터를 많이 학습할수록) 용기는 더 가득 차게 되고(모델의 정확도가 높아집니다).

      * **다양한 학습 방법:**

      * 위에서 설명한 것은 지도 학습(Supervised Learning)의 기본 원리입니다.  이 외에도 비지도 학습(Unsupervised Learning), 강화 학습(Reinforcement Learning) 등 다양한 학습 방법이 존재합니다.  각 방법은 데이터의 종류와 학습 목표에 따라 적절하게 선택됩니다.

      * 결론적으로, AI 모델은 데이터를 통해 패턴을 학습하고, 오류를 수정하며 점점 더 정확한 예측을 할 수 있도록 발전하는 것입니다.  이는 반복적인 과정을 통해 이루어지며,  데이터의 양과 질이 모델의 성능에 큰 영향을 미칩니다.
  
      * ✅ invoke() 호출 성공.

    * --- stream() 호출 결과 (직접 스트리밍 출력) ---
      * 인공지능 모델의 학습 원리는 마치 어린아이가 배우는 것과 비슷합니다.  아이가 "강아지"라는 단어와 강아지 사진을 여러 번 반복해서 보면서 "강아지"라는 단어가 무엇을 의미하는지 배우는 것처럼, AI 모델도 데이터를 통해 학습합니다.

      * 핵심은 **데이터**와 **피드백**입니다.

        * 1. **데이터 입력:**  AI 모델은 방대한 양의 데이터(예: 강아지 사진, 고양이 사진, 설명 등)를 입력받습니다.  이 데이터는 모델이 학습할 "교재"와 같습니다.

        * 2. **모델의 예측:**  입력된 데이터를 바탕으로 모델은 스스로 "강아지"와 "고양이"를 구분하는 방법을 찾으려고 합니다. 처음에는 엉뚱한 예측을 할 수도 있습니다.  마치 아이가 처음에는 강아지와 고양이를 구분하지 못하는 것과 같습니다.

        * 3. **피드백 (오류 수정):**  모델의 예측이 정답과 맞지 않으면 (예: 강아지를 고양이로 잘못 분류),  피드백을 통해 오류를 수정합니다.  이 피드백은 모델의 "선생님" 역할을 합니다.  모델은 이 피드백을 통해 자신의 예측 방식을 조정하고, 다음에는 더 정확한 예측을 하도록 "학습"합니다.  이 과정을 반복하면서 모델의 정확도가 점점 높아집니다.

        * 4. **가중치 조정:**  모델은 내부적으로 수많은 "가중치"라는 값들을 가지고 있습니다.  이 가중치는 데이터의 특징(예: 귀의 모양, 털의 색깔)에 대한 중요도를 나타냅니다.  피드백을 통해 모델은 이 가중치를 조정하여 더 정확한 예측을 할 수 있도록 합니다.  이 과정은 마치 아이가 강아지의 특징(귀, 꼬리, 털 등)을 점점 더 잘 인식하게 되는 것과 같습니다.

        * 5. **반복 학습:**  1~4단계를 수없이 반복하면서 모델은 점점 더 정확하게 강아지와 고양이를 구분할 수 있게 됩니다.  이 과정을 **훈련(Training)**이라고 합니다.

      * 간단히 말해, AI 모델은 데이터를 통해 패턴을 학습하고, 피드백을 통해 오류를 수정하며, 점점 더 정확한 예측을 할 수 있도록 스스로 발전하는 것입니다.  이러한 학습 방식은 다양한 종류의 AI 모델에 적용되지만, 기본 원리는 모두 동일합니다.

      * ✅ stream() 호출 성공.

---

#### (5) 템플릿 변경 적용

- 아래의 프롬프트 내용을 얼마든지 **변경** 하여 테스트 가능
- `model_name` 역시 변경하여 테스트 가능

In [13]:
# 템플릿 변경하여 적용 
print("\n--- 템플릿 변경 적용 예시 ---")

# 템플릿 변경
template_english_teacher = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 상황에 [FORMAT]에 영어 회화를 작성해 주세요.

상황:
{question}

FORMAT:
- 영어 회화:
- 한글 해석:
"""

# 프롬프트 생성
prompt_english_teacher = PromptTemplate.from_template(template_english_teacher)

# 모델 변경 (gemini-1.5-flash -> gemini-1.5pro)
try:
    model_pro = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash-lite",                      # 모델 변경
        temperature=0.1,
        api_key=os.getenv("GOOGLE_API_KEY")                 # GOOGLE_API_KEY 환경 변수 불러오기
    )
    print("✅ Google GenAI 'gemini-1.5-pro' 모델 초기화 성공.")
except Exception as e:
    print(f"❌ Google GenAI 'gemini-1.5-pro' 모델 초기화 실패: {e}")
    print("  -> GOOGLE_API_KEY 환경 변수가 올바르게 설정되었는지 확인하세요.")
    # model_pro가 정의되지 않을 경우를 대비하여 오류 발생 시 강제 종료 (선택 사항)
    # exit() 또는 raise e

# 새로운 체인 구성: prompt_english_teacher + 새로 정의한 model_pro 연결
chain_english_teacher = prompt_english_teacher | model_pro | output_parser
print("✅ LangChain LCEL '영어 선생님' 체인 (gemini-1.5-pro) 구성 완료.")


print("\n--- 영어 회화 스트리밍 출력 ---")
try:
    answer_english_teacher = chain_english_teacher.stream({"question": "저는 식당에 가서 음식을 주문하고 싶어요"})
    stream_and_print_response(answer_english_teacher)
    print("✅ 영어 회화 스트리밍 호출 성공.")
except Exception as e:
    print(f"❌ 영어 회화 스트리밍 중 오류 발생: {e}")

print("\n--- 미국에서 피자 주문 스트리밍 출력 ---")
try:
    answer_pizza_order = chain_english_teacher.stream({"question": "미국에서 피자 주문"})
    stream_and_print_response(answer_pizza_order)
    print("✅ 피자 주문 스트리밍 호출 성공.")
except Exception as e:
    print(f"❌ 피자 주문 스트리밍 중 오류 발생: {e}")


--- 템플릿 변경 적용 예시 ---
✅ Google GenAI 'gemini-1.5-pro' 모델 초기화 성공.
✅ LangChain LCEL '영어 선생님' 체인 (gemini-1.5-pro) 구성 완료.

--- 영어 회화 스트리밍 출력 ---
네, 10년차 영어 선생님으로서 식당에서 음식을 주문하는 상황에 맞는 영어 회화를 작성해 드리겠습니다.

---

**상황:** 식당에 가서 음식을 주문하고 싶어요.

**FORMAT:**

*   **영어 회화:**

    **You:** Hello! Table for two, please.
    **Waiter/Waitress:** Good evening! Right this way. Here are your menus.
    **You:** Thank you.
    *(A few minutes later)*
    **Waiter/Waitress:** Are you ready to order?
    **You:** Yes, I think so. I'll have the grilled salmon, please.
    **Waiter/Waitress:** Excellent choice. And for you?
    **You:** I'll have the chicken stir-fry.
    **Waiter/Waitress:** Okay. And would you like anything to drink?
    **You:** Yes, I'll have a glass of water, please.
    **Waiter/Waitress:** Anything else?
    **You:** No, that's all for now, thank you.
    **Waiter/Waitress:** Alright. Your order will be out shortly.

*   **한글 해석:**

    **당신:** 안녕하세요! 두 명 자리 부탁드립니다.
    **웨이터/웨이트리스:** 

<small>

* 셀 출력
  
    * --- 템플릿 변경 적용 예시 ---
    * ✅ Google GenAI 'gemini-1.5-pro' 모델 초기화 성공.
    * ✅ LangChain LCEL '영어 선생님' 체인 (gemini-1.5-pro) 구성 완료.

---

    * --- 영어 회화 스트리밍 출력 ---
    * 네, 10년차 영어 선생님으로서 식당에서 음식을 주문하는 상황에 맞는 영어 회화를 작성해 드리겠습니다.

    ---

    * **상황:** 식당에 가서 음식을 주문하고 싶어요.

    * **FORMAT:**

    *   **영어 회화:**

        * **You:** Hello! Table for two, please.
        * **Waiter/Waitress:** Good evening! Right this way. Here are your menus.
        * **You:** Thank you.
        * *(A few minutes later)*
        * **Waiter/Waitress:** Are you ready to order?
        * **You:** Yes, I think so. I'll have the grilled salmon, please.
        * **Waiter/Waitress:** Excellent choice. And for you?
        * **You:** I'll have the chicken stir-fry.
        * **Waiter/Waitress:** Okay. And would you like anything to drink?
        * **You:** Yes, I'll have a glass of water, please.
        * **Waiter/Waitress:** Anything else?
        * **You:** No, that's all for now, thank you.
        * **Waiter/Waitress:** Alright. Your order will be out shortly.
 
    *   **한글 해석:**

        * **당신:** 안녕하세요! 두 명 자리 부탁드립니다.
        * **웨이터/웨이트리스:** 안녕하세요! 이쪽으로 오세요. 여기 메뉴입니다.
        * **당신:** 감사합니다.
        * *(몇 분 후)*
        * **웨이터/웨이트리스:** 주문하시겠어요?
        * **당신:** 네, 그런 것 같아요. 저는 구운 연어를 먹겠습니다.
        * **웨이터/웨이트리스:** 훌륭한 선택입니다. 그리고 당신은요?
        * **당신:** 저는 치킨 볶음을 먹겠습니다.
        * **웨이터/웨이트리스:** 알겠습니다. 음료는 뭘로 하시겠어요?
        * **당신:** 네, 물 한 잔 주세요.
        * **웨이터/웨이트리스:** 더 필요하신 건 없으신가요?
        * **당신:** 아니요, 지금은 이게 전부입니다. 감사합니다.
        * **웨이터/웨이트리스:** 네. 곧 음식이 나올 겁니다.

    ---

    * **추가 팁:**

    *   **메뉴를 볼 때:** "Could you recommend something?" (무엇을 추천해 주시겠어요?) 또는 "What's your specialty?" (이 집의 특별 메뉴가 무엇인가요?) 라고 물어볼 수 있습니다.
    *   **음식에 대한 질문:** "Is the pasta spicy?" (이 파스타 매콤한가요?) 또는 "Does the steak come with any sides?" (스테이크에 곁들여 나오는 것이 있나요?) 와 같이 궁금한 점을 물어볼 수 있습니다.
    *   **주문 변경:** "Could I have the salad dressing on the side?" (샐러드 드레싱을 따로 주실 수 있나요?) 와 같이 요청할 수 있습니다.
    *   **계산할 때:** "Could we have the bill, please?" (계산서 좀 주시겠어요?) 라고 말하면 됩니다.

    * 이 대화가 식당에서 자신감 있게 주문하는 데 도움이 되기를 바랍니다! 혹시 다른 상황이나 더 궁금한 점이 있다면 언제든지 물어보세요.
    * ✅ 영어 회화 스트리밍 호출 성공.

---

    * --- 미국에서 피자 주문 스트리밍 출력 ---
    * ## 영어 회화 연습: 미국에서 피자 주문하기

    * 안녕하세요! 10년차 영어 선생님입니다. 오늘은 미국에서 피자를 주문하는 상황을 연습해 볼 거예요. 실제 상황처럼 자연스럽게 대화할 수 있도록 함께 연습해 봅시다!

    ---

    * **상황:** 미국에서 피자 가게에 전화해서 피자를 주문하는 상황

    * **FORMAT:**

    * **영어 회화:**

        * **You:** Hello, I'd like to order a pizza for delivery, please.
        * **Pizza Place:** Hi there! Sure, what can I get for you today?
        * **You:** I'd like a large pepperoni pizza.
        * **Pizza Place:** Okay, a large pepperoni. Anything else?
        * **You:** Yes, could I also get a side of garlic knots?
        * **Pizza Place:** Absolutely. Garlic knots, got it. And for the delivery address?
        * **You:** It's 123 Main Street, Apartment 4B.
        * **Pizza Place:** 123 Main Street, Apartment 4B. And a phone number for the order?
        * **You:** My number is 555-123-4567.
        * **Pizza Place:** Great. So that's one large pepperoni pizza and a side of garlic knots, going to 123 Main Street Apartment 4B. Your total will be $25.50. It should arrive in about 45 minutes.
        * **You:** Sounds good. Thank you!
        * **Pizza Place:** You're welcome! Enjoy your pizza!

    * **한글 해석:**

        * **당신:** 안녕하세요, 배달 피자를 주문하고 싶습니다.
        * **피자 가게:** 안녕하세요! 네, 오늘 무엇을 주문하시겠어요?
        * **당신:** 라지 사이즈 페퍼로니 피자 하나 주세요.
        * **피자 가게:** 네, 라지 페퍼로니 피자. 다른 건요?
        * **당신:** 네, 마늘 빵도 하나 추가할 수 있을까요?
        * **피자 가게:** 물론이죠. 마늘 빵, 알겠습니다. 배달 주소는 어떻게 되시나요?
        * **당신:** 123 메인 스트리트, 아파트 4B입니다.
        * **피자 가게:** 123 메인 스트리트, 아파트 4B. 주문하신 분 연락처는요?
        * **당신:** 제 번호는 555-123-4567입니다.
        * **피자 가게:** 좋습니다. 그럼 라지 페퍼로니 피자 하나와 마늘 빵 하나, 123 메인 스트리트, 아파트 4B로 배달입니다. 총 금액은 25.50달러입니다. 약 45분 후에 도착할 예정입니다.
        * **당신:** 좋습니다. 감사합니다!
        * **피자 가게:** 천만에요! 피자 맛있게 드세요!

    ---

    * **추가 팁:**

    *   **피자 종류:** "pepperoni" 대신 "margherita", "veggie", "supreme" 등 원하는 피자 종류를 말할 수 있습니다.
    *   **사이즈:** "large" 대신 "medium", "small"을 사용할 수 있습니다.
    *   **토핑 추가/제외:** "Can I add extra cheese?" (치즈 추가할 수 있나요?) 또는 "Can I get it without onions?" (양파 빼고 주실 수 있나요?) 와 같이 요청할 수 있습니다.
    *   **음료:** "Could I also get a 2-liter bottle of Coke?" (콜라 2리터 병도 하나 주실 수 있나요?) 와 같이 음료를 주문할 수도 있습니다.
    *   **결제:** 보통 배달 시 현금 또는 카드 결제가 가능합니다. "I'll pay with cash." (현금으로 결제할게요.) 또는 "I'll pay with credit card." (카드로 결제할게요.) 라고 말할 수 있습니다.

    * 이 대화를 여러 번 반복해서 연습해 보세요. 실제 상황에서 자신감 있게 피자를 주문할 수 있을 거예요! 궁금한 점이 있다면 언제든지 질문해주세요!
    * ✅ 피자 주문 스트리밍 호출 성공.